# Processing the 'PlateMicroscopy' directory
__Fall 2019__<br>
__Keith Cheveralls__

This notebook documents the development of methods to process the raw TIFFs found in the 'PlateMicroscopy' directory. This directory contains all raw pipeline/opencell microscopy data up to PML0179; this data was acquired by Nathan/Preethi and used Nathan's automation scripts (these were either beanshell scripts or, later, mm2python-powered python scripts). It corresponds to all datasets for Plates 1-19 and for thawed Plates 1-5. This data was acquired between October 2018 and August 2019. 

It also includes analysis of the FOV scores for all of the FOVs in this directory. 

In [ ]:
import os
import re
import sys
import glob
import json
import nrrd
import shutil
import pickle
import hashlib
import skimage
import datetime
import tifffile
import numpy as np
import pandas as pd

import imageio
import dask
import dask.diagnostics

from matplotlib import pyplot as plt

sys.path.append('..')
from opencell.database import models, operations
from opencell.database import utils as db_utils

sys.path.append('/Users/keith.cheveralls/projects/dragonfly-automation/')
import dragonfly_automation.utils
from dragonfly_automation.fov_models import PipelineFOVScorer

In [ ]:
%load_ext autoreload
%autoreload 1

%aimport opencell.imaging.processors
%aimport opencell.imaging.managers
%aimport opencell.imaging.images

from opencell.imaging import images, utils, viz
from opencell.imaging.managers import PlateMicroscopyManager
from opencell.imaging.processors import FOVProcessor
from opencell.imaging.images import RawPipelineTIFF

In [ ]:
# ESS 'PlateMicroscopy' directory
ess_root = '/Volumes/ml_group/PlateMicroscopy/'
os.path.isdir(ess_root)

dst_root = '/Volumes/ml_group/opencell-microscopy/'
os.path.isdir(dst_root)

# Parsing and processing all raw FOVs

### Instance of a PlateMicroscopy API

In [ ]:
pm = PlateMicroscopyManager(ess_root, '../plate-microscopy-cache/20191114-ess/')
len(pm.os_walk), pm.md.shape[0], pm.md.is_raw.sum(), pm.md_raw.shape[0]

In [ ]:
pm.md_raw

### Parsing the metadata text files

As far as I can tell, there's nothing in these text files (which are actually JSON files) that's not also in the IJMetadata and MicroManagerMetadata TIFF tags.

In [ ]:
with open(pm.src_filepath(d_raw.iloc[0]).replace('.ome.tif', '_metadata.txt'), 'r') as file:
    d = json.load(file)

In [ ]:
sorted([(key, val) for key, val in d['FrameKey-0--1-0'].items()])

### Parsing raw TIFF metadata


In [ ]:
t = RawPipelineTIFF('/Users/keith.cheveralls/image-data/MMStack_0-B9-0.ome.tif')

In [ ]:
t.parse_micromanager_metadata()
t.validate_micromanager_metadata()

In [ ]:
t.split_channels()

### Summary of metadata issues

__Loading tiffs with `tifffile.TiffFile`__<br>
The stand-alone tifffile package (v0.15.1) works to load all raw TIFFs. There are 14754 stacks in 'v1' metadata format and 5243 in 'v2' format.


__Inconsistent number of slices per channel__
- 'P0014_ML0118_E2_1_RPS6KA4_events' page 50 - last page missing tags and the GFP channel is missing completely
- 'P0014_ML0120_H5_12_VRK3_events' page 76 - last page is missing tags and half of the GFP channel is missing
- 'P0018_ML0132_F4_4_GOLT1B_events' page 192 - last page missing tags and uneven number of slices in DAPI and GFP
 
__Inconsistent exposure times__<br>
There are three TIFFs with inconsistent exposure times: 'G5_22_TRIM24.ome.tif', 'G12_13_ANLN.ome.tif', 'F9_9_JAK1.ome.tif'. In all cases, the exposure time from the GFP seems to have been prematurely assigned to some of the DAPI slices. TODO: determine whether this is true for the metadata or the actual acqusitions.  


__Other issues__<br>
- some raw TIFFs have a negative DAPI channel index (indices are -1 and 0 for DAPI and GFP)
- some raw TIFFs have an extra page with no metadata or data
- some raw TIFFs may have extra pages at the end, possibly with valid metadata (according to Nathan)

__Missing metadata in disentangled stacks for Plate16,17,18__<br>
The raw TIFFs from Plates 16,17,18 that were disentangled from 'giant' stacks in the `_compressed` subdirectories using Nathan's `stackDisentangle.py` script all have invalid MM metadata tags. Due to a bug, the MM metadata tag from the first page of the disentangled stack appears on every page. Retrieving the true MM metadata for each page will require re-disentangling the stacks. 

### Load all parsing events and the parsed metadata

Note that this is deprecated until we implement a new method to aggregate all of the processing-event CSVs (which are not inserted into the database).

In [ ]:
ev = pd.read_csv(os.path.join(dst_root, 'aggregated-processing-events.csv'))

In [ ]:
ev = ev.loc[ev.message.apply(lambda m: 'IJMetadata' not in m)]
ev = ev.loc[ev.message.apply(lambda m: 'Inconsistent values' not in m)]
ev.groupby(['plate_dir', 'message']).agg(['count', 'first'])

In [ ]:
df = pd.read_csv('%s/aggregated-raw-tiff-metadata.csv' % dst_root)

In [ ]:
_ = plt.hist(df.gfp_exposure_time, bins=np.arange(0, 400, 1))

In [ ]:
counts, edges = np.histogram(df.gfp_max_intensity, bins=np.arange(0, 65535, 100))
plt.plot(edges[1:], (counts + 1))

In [ ]:
plt.scatter(df.gfp_exposure_time, df.gfp_max_intensity, alpha=.1)

In [ ]:
(df.gfp_max_intensity==65535).sum()

In [ ]:
df.gfp_exposure_time.isna().sum()

# Scoring all raw FOVs

In [ ]:
scorer = PipelineFOVScorer(mode='training', model_type='regression')
scorer.load('/Users/keith.cheveralls/projects/dragonfly-automation/models/2019-10-08/')
scorer.train()

### Data for all raw FOVs from database

In [ ]:
url = db_utils.url_from_credentials('../db-credentials-cap.json')

In [ ]:
# load features and score for all FOVs from opencelldb
result_kind = 'calculate_fov_features'
with operations.session_scope(url) as session:
    results = session.query(models.MicroscopyFOVResult)\
        .filter(models.MicroscopyFOVResult.method_name==result_kind).all()
    dfeat = pd.DataFrame(
        data=[{'line_id': row.fov.cell_line_id, 'fov_id': row.fov.id, **row.data} for row in results])

In [ ]:
dfeat.head()

In [ ]:
# instantiate FOV processors from opencelldb
with operations.session_scope(url) as session:
    fovs = session.query(models.MicroscopyFOV).all()
    ps = [FOVProcessor.from_database(fov) for fov in fovs]

In [ ]:
# construct FOV metadata (fov_id, plate_id, well_id, imaging_round_id)
rows = []
for p in ps:
    filepath = p.dst_filepath(
        dst_root='/Volumes/ml_group/opencell-microscopy/', 
        kind='projection', 
        channel='dapi', 
        axis='z',
        makedirs=False)

    filepath = p.tag_filepath(filepath, tag='DAPI-PROJ-Z', ext='tif')
    rows.append({
        'fov_id': p.fov_id,
        'filepath': filepath,
        'plate_id': p.plate_id,
        'well_id': p.well_id,
        'imaging_round_id': p.imaging_round_id,
        'target_name': p.target_name,
    })

In [ ]:
dmeta = pd.DataFrame(data=rows)
dmeta.head()

In [ ]:
# merge metadata and features on fov_id
data = pd.merge(dmeta, dfeat, left_on='fov_id', right_on='fov_id', how='inner')
data.shape

In [ ]:
# columns used in plots below
data['yp'] = data.score
data['plate_num'] = [int(s[1:]) for s in data.plate_id]
data['imaging_round_num'] = [int(s[1:]) for s in data.imaging_round_id]

In [ ]:
len(set(data.target_name))

In [ ]:
# number of cell_lines without any score-able FOVs
data.groupby('line_id').score.max().isna().sum()

In [ ]:
# test a processor of one FOV
p = [p for p in ps if p.fov_id==18000][0]
p.dst_filepath('', kind='cropz')

In [ ]:
p.crop_cell_layer('/Volumes/ml_group/PlateMicroscopy/', dst_root)

In [ ]:
p.generate_nrrd(dst_root)

### Data for all raw FOVs from cached results CSVs

This is after calculating features from the z-projections of all raw FOVs using the `imaging_tasks` CLI on `cap`. 

In [ ]:
all_md = pd.read_csv('/Volumes/ml_group/oc-plate-microscopy/aggregated-raw-tiff-metadata.csv')

# patch fov_id ('ML0125-C11-4' -> 'ML0125-C11-S04') (change made on Fri 2019-11-15)
all_md['fov_id'] = [
    '-'.join(fov_id.split('-')[:-1] + ['S%02d' % int(fov_id.split('-')[-1])])
    for fov_id in all_md.fov_id]
all_md.shape

In [ ]:
len(set(all_md.fov_id))

In [ ]:
all_features = pd.read_csv('/Volumes/ml_group/oc-plate-microscopy/aggregated-fov-features.csv')

# patch fov_id ('ML0125-C11-4' -> 'ML0125-C11-S04') (change made on Fri 2019-11-15)
all_features['fov_id'] = [
    '-'.join(fov_id.split('-')[:-1] + ['S%02d' % int(fov_id.split('-')[-1])])
    for fov_id in all_features.fov_id]
all_features.shape

In [ ]:
# merge data and metadata
data = pd.merge(pm.md_raw.copy(), all_features, left_on='fov_id', right_on='fov_id', how='inner')
data.rename(columns={'filename_x': 'src_filename', 'filename_y': 'dst_filename'}, inplace=True)
data.shape, pm.md_raw.shape

In [ ]:
data.groupby('error').count().is_raw

In [ ]:
data = data.loc[data.error.isna()]
data.drop(labels='error', axis=1, inplace=True)
data.shape

In [ ]:
# construct filepaths to the DAPI z-projections
def construct_filepath(filename, dst_root):
    plate_dir = '-'.join(filename.split('-')[:4])
    filepath = os.path.join(dst_root, plate_dir, filename)
    return filepath

dst_root = '/Users/keith.cheveralls/image-data/oc-plate-microscopy/projections/dapi/z'
data['filepath'] = None
for ind, row in data.iterrows():
    data.at[ind, 'filepath'] = construct_filepath(row.dst_filename, dst_root)

In [ ]:
# force inf to nan because dropna does not drop np.infs
data = data.replace([np.inf, -np.inf], np.nan)

# drop FOVs with missing features
data = data.dropna(axis=0, how='any', subset=scorer.feature_order)
data.shape

In [ ]:
# predicted scores for unsorted FOVs
X = data[list(scorer.feature_order)].values
yp = scorer.model.predict(X)
data['yp'] = yp

### Summaries and plots of FOV scores

In [ ]:
# basic summary by well
dg = data.groupby(['plate_id', 'well_id', 'target_name'])
summary = pd.concat((dg.yp.max(), dg.yp.count()), axis=1)
summary.columns = 'max_score', 'num_fovs'

summary = summary.sort_values(by=['plate_id', 'max_score'])
# summary.to_csv('/Users/keith.cheveralls/image-data/oc-plate-microscopy-max-scores.csv')

In [ ]:
# distribution of max score per target
plt.hist(data.groupby('target_name').yp.max().values, bins=20)

### Tile all FOVs, sorted by score, from a particular plate

In [ ]:
# FOVs from a particular plate with a particular score
d = data.loc[(data.plate_num==8) & (data.imaging_round_num==1)].copy()
d = d.sort_values(by='yp', ascending=False)
d.shape, 25*25

In [ ]:
# sanity check - manually verify that the target_names for the selected plate 
# match the 'mNG11 HEK Library' google sheet
sorted(d.target_name.unique())

In [ ]:
# sanity check - manually verify the ML experiment IDs are correct
sorted(d.exp_id.unique())

In [ ]:
bad = (d.yp < -.5)
neutral = (d.yp > -.5) & (d.yp < .5)
good = d.yp > .5
alll = bad | neutral | good

tile = viz.build_tile(
    d.loc[alll], 
    shape=(30, 30), 
    figsize=25, 
    offset=0,
    show_labels=True, 
    label_column='yp', 
    label_format='%0.2f')

In [ ]:
tifffile.imsave('/Users/keith.cheveralls/image-data/all-raw-plate7-ordered-by-score-30x30.tif', tile)

In [ ]:
# make tiles for all plates
for plate_id in data.plate_id.unique():
    print(plate_id)
    d = data.loc[data.plate_id==plate_id].copy()
    d = d.sort_values(by='yp', ascending=False)
    tile = viz.build_tile(
        d, 
        shape=(30, 30), 
        figsize=25, 
        offset=0,
        plot=False)
    tifffile.imsave('/Users/keith.cheveralls/image-data/FOV-tile-all-raw-plate%s-30x30.tif' % plate_id, tile)

### Histograms of scores per plate and imaging round

In [ ]:
plate_round_nums = [(n, 1) for n in range(1, 21)] + [(n, 2) for n in range(1, 6)]

bin_width = 0.2
plot_neg = True
if plot_neg:
    bin_min = -1.0
    ymax = 4
else:
    bin_min = -.8
    ymax = 1.5

n = 0
fig, axs = plt.subplots(5, 5, figsize=(16, 12))
for rind, row in enumerate(axs):
    for cind, ax in enumerate(row):
        plate_num, round_num = plate_round_nums[n]
        n += 1
        if plate_num > 19:
            continue
            
        values = data.loc[(data.plate_num==plate_num) & (data.imaging_round_num==round_num)].yp.values
        ax.hist(values, bins=np.arange(bin_min, 1 + bin_width, bin_width), density=True)
        ax.set_title('Plate %s-%s (n = %d)' % (plate_num, round_num, len(values)))    

        ax.set_ylim([0, ymax])
        ax.set_xticks([-1, -.5, 0, .5, 1])
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        if cind==0:
            ax.set_ylabel('Density')
            ax.set_yticklabels([0, 1, 2, 3])
        if rind==len(axs)-1:
            ax.set_xlabel('Score')
            ax.set_xticklabels([-1, -.5, 0, .5, 1])

### Percent good/bad FOVs

In [ ]:
# percent good and bad scores per plate-round
pbad, pgood = [], []
for plate_round_num in plate_round_nums:
    plate_num, round_num = plate_round_num
    values = data.loc[(data.plate_num==plate_num) & (data.imaging_round_num==round_num)].yp.values
    pgood.append((values > .7).sum() / len(values))
    pbad.append((values < -.7).sum() / len(values))

In [ ]:
# percent good and bad *max* scores (by target_name), per plate-round
pbad, pgood, pnan = [], [], []
for plate_round_num in plate_round_nums:
    plate_num, r_num = plate_round_num
    values = data.loc[(data.plate_num==plate_num)].groupby('target_name').max().yp.values
    pgood.append((values > .5).sum() / len(values))
    pbad.append((values < -.5).sum() / len(values))
    pnan.append(np.isnan(values).sum() / len(values))

In [ ]:
fig, ax = plt.subplots(figsize=(12, 3))
width = 1/4

x = np.arange(len(plate_round_nums))
rects1 = ax.bar(x - width, np.array(pgood)*100, width, label='Predicted good')
rects2 = ax.bar(x + 0*width, np.array(pbad)*100, width, label='Predicted bad')
rects3 = ax.bar(x + width, np.array(pnan)*100, width, label='All NaN')

ax.set_ylabel('Percent')
ax.set_title('')
ax.set_xticks(x)
ax.set_xticklabels(['%s-%s' % plate_round_num for plate_round_num in plate_round_nums])
ax.legend()

# Recapitulate Nathan's stack disentangling script

We see that TiffWriter.save does not actually save the MM metadata tag for each page; instead, it saves the MM tag from the first page with every subsequent page.

In [ ]:
t = images.RawPipelineTIFF('/Users/keith.cheveralls/image-data/plate18-ex-compressed/MMStack_31.ome.tif')

In [ ]:
t = images.RawPipelineTIFF('/Users/keith.cheveralls/image-data/plate17-ex-compressed/MMStack_0.ome.tif')

In [ ]:
t.parse_micromanager_metadata()
t.validate_micromanager_metadata()

In [ ]:
plt.plot(t.mm_metadata.slice_ind)

In [ ]:
entangled_tiff = tifffile.TiffFile('/Users/keith.cheveralls/image-data/plate17-ex-compressed/MMStack_0.ome.tif')

new_pages = []
new_tags = []
for ind in range(222):
    page = entangled_tiff.pages[ind]
    new_pages.append(page.asarray())
    mm_metadata = json.dumps(page.tags['MicroManagerMetadata'].value)
    mm_tag = ('MicroManagerMetadata', 's', 0, mm_metadata, False)
    new_tags.append(mm_tag)

In [ ]:
t_out = tifffile.TiffWriter('/Users/keith.cheveralls/image-data/plate17-ex-compressed/test-disentangle-first-222-pages.tif')
for page, tag in zip(new_pages, new_tags):
    t_out.save(page, extratags=[tag], contiguous=False)
t_out.close()

In [ ]:
t = images.RawPipelineTIFF('/Users/keith.cheveralls/image-data/plate17-ex-compressed/test-disentangle-first-222-pages.tif')

In [ ]:
t.parse_micromanager_metadata()
t.validate_micromanager_metadata()

### Nathan's QC CSVs

In [ ]:
qc14 = pd.concat([
    pd.read_csv(f, header=None, names=['target', 'flag']) 
    for f in glob.glob('/Users/keith.cheveralls/Box/AutomatedImageQC/QCFiles/Plate14/*.csv')])

qc15 = pd.concat([
    pd.read_csv(f, header=None, names=['target', 'flag']) 
    for f in glob.glob('/Users/keith.cheveralls/Box/AutomatedImageQC/QCFiles/Plate15/*.csv')])

In [ ]:
qc14.loc[qc14.flag=='GoodStack'].shape, qc15.loc[qc15.flag=='GoodStack'].shape

In [ ]:
qc14.shape, qc15.shape